In [2]:
from guidance import models, gen, select
import guidance
lm = models.Transformers('TheBloke/Nous-Hermes-2-SOLAR-10.7B-GPTQ', device_map="cuda", echo=True, temperature=50, max_length = 10)


In [2]:
def yield_test(n):
    print("start n =", n)
    for i in range(n):
        yield i*i
        print("i =", i)
        if i == 2:  # Add a condition to return early
            return
    print("end")

tests = yield_test(5)
for test in tests:
    print("test =", test)
    print("--------")

start n = 5
test = 0
--------
i = 0
test = 1
--------
i = 1
test = 4
--------
i = 2


In [4]:
lm += "Hello! I "+gen(max_tokens=10, temperature=50)

In [ ]:

import chromadb
from sentence_transformers import SentenceTransformer
#sentences = ["This is an example sentence", "Each sentence is converted"]

sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

import time
from datetime import date

In [ ]:

#embeddings = sentence_transformer.encode(sentences)#
short_term_memory = chromadb.Client()
#client = chromadb.PersistentClient(path="./Chroma")
working_memory = []
"""
This is what a Info Block should look like
{
	'object_type' : 'people',
	'object_name' : 'Miko',
	'content' : 'Miko like Nintendo games.',
	'timestamp' : '2021-10-15 17:37:00',
	'vector' array([[-4.39221077e-02, -1.25277145e-02,  2.93133650e-02,]], dtype=float32): ,
}    

with guidance
f'''\
{
	'object_type' : '{select(['people','events'])'},
	'object_name' : '{gen(stop='\'')},
	'content' : '{gen(stop='\'')},
	'timestamp' : {date.fromtimestamp(time.time())},
	'vector' : {sentence_transformer.encode([])},
}'''
"""
#def process_new_info_block(lm_text_result):

@guidance(stateless=True)
def guidance_make_new_info_block(lm, passage):
		first_curly = "{"
		second_curly = "}"
		lm += """\
This is what a Info Block should look like from an example:
[Example]
Lucid: What are you doing now? Stop ignoring me!
Miko: I'm playing Nintendo games.
Lucid: And that's a higher priority than me?
Miko: ...
Lucid: Your silence speaks volumes.

```json
{
	"object_type" : 'people',
	"object_name" : 'Miko',
	"content" : 'Miko likes Nintendo games.',
}
```
[End of Example]
"""
		lm += f"""\
{passage}
```json
{first_curly}
	"object_type" : '{select(['people','events'], name="object_type")},
	"object_name" : "{gen(stop='"',name = "object_name")},
	"content" : "{gen(stop='"',name="content")},"""  
		return lm

def make_new_info_block(lm, passage):
	first_curly = "{"
	second_curly = "}"
	lm += guidance_make_new_info_block(passage)
	print(lm['object_type'],lm['object_name'],lm['content'])
	timestamp = date.fromtimestamp(time.time())
	json_file ={
	"object_type" : lm['object_type'],
	"object_name" : lm['object_name'],
	"content" : lm['content'],
	"timestamp" : timestamp,
	"vector" : sentence_transformer.encode([lm['content']]),
	}
	return json_file

In [ ]:
passage = """\
Lucid: Hey Miko what's your favorite food?
Miko: Umm I don't know maybe rice?
Lucid you are really boring you know?"""
make_new_info_block(lm, passage)

In [ ]:
import sys
import traceback


class Item:
    SPACE = ' '
    DASH_SPACE = ' - '
    def __init__(self, key, name, function, parent=None):
        self.key = str(key)
        self.name = name
        self.function = function
        self.parent = parent

        if parent:
            parent.add_item(self)

    def __str__(self):
        return self.SPACE + self.key + self.DASH_SPACE + self.name

    def execute(self):
        print('\n' + self.name + '\n')
        return self.function()


class Menu:
    def __init__(self, name, items=None):
        self.name = name
        self.items = items or []

    def add_item(self, item):
        self.items.append(item)
        if item.parent != self:
            item.parent = self

    def remove_item(self, item):
        self.items.remove(item)
        if item.parent == self:
            item.parent = None

    def __str__(self):
        all = ['', self.name, '']
        for item in self.items:
            all.append(str(item))
        return '\n'.join(all)
            
    def get_item(self, key):
        for item in self.items:
            if item.key == key:
                return item

    def execute(self):
        while True:
            try:
                print(self)
                choice = raw_input(" >>  ")
                try:
                    item = self.get_item(choice)

                    try:
                        item.execute()
                    except Exception:
                        traceback.print_exc()

                except Exception:
                    print('Invalid selection, please try again.\n')
            except KeyboardInterrupt:
                sys.exit()
    
    
if __name__ == "__main__":
    # Pass in methods to run for each menu item
    menu_items = [
        Item(1, 'View Running tasks', view_running_tasks),
        Item(2, 'View Task', view_task),
        # ...
        Item(9, 'Exit', exit)]
    menu = Menu('Task Management', items=menu_items)

    menu.execute()